In [5]:
import requests
from bs4 import BeautifulSoup

In [6]:
# The URL that provides the raw alert data for the iframe
# This corresponds to the "Alle Regio's" view
url = 'http://www.p2000-online.net/p2000.py'

In [ ]:
print(f"Scraping data from: {url}")

In [22]:
try:
    # Make a request to the website
    response = requests.get(url)
    response.raise_for_status()

    # Explicitly set the correct character encoding
    response.encoding = 'windows-1252'

    # Use the corrected content for parsing
    soup = BeautifulSoup(response.text, 'html.parser')

    all_alerts_raw = []
    
    # Find all potential main alert rows
    main_alert_rows = soup.find_all(lambda tag: tag.name == 'tr' and tag.find('td', class_='DT'))

    for row in main_alert_rows:
        # Extract data from the main alert row
        dt_cell = row.find('td', class_='DT')
        service_cell = row.find('td', class_=['Am', 'Br', 'Po'])
        region_cell = row.find('td', class_='Regio')
        message_cell = row.find('td', class_=['Md', 'Mdx'])

        # Ensure all main cells are present before adding
        if dt_cell and service_cell and region_cell and message_cell:
            alert_data = {
                "datetime": dt_cell.text.strip(),
                "service": service_cell.text.strip(),
                "region": region_cell.text.strip(),
                "message": message_cell.text.strip(),
            }
            all_alerts_raw.append(alert_data)

    # De-duplication Step
    unique_alerts = []
    seen_alerts = set()

    for alert in all_alerts_raw:
        # Create a unique identifier for each alert (timestamp + message)
        identifier = (alert['datetime'], alert['message'])
        
        # If we haven't seen this alert before, add it to our unique list
        if identifier not in seen_alerts:
            unique_alerts.append(alert)
            seen_alerts.add(identifier)

    # --- MODIFIED: Print only the latest alert ---
    if not unique_alerts:
        print("\nScraping finished, but no alerts were found.")
    else:
        # The first alert in the list is the latest one
        latest_alert = unique_alerts[0]
        
        print("\n--- Latest Alert ---")
        print(f"Time:    {latest_alert['datetime']}")
        print(f"Service: {latest_alert['service']}")
        print(f"Region:  {latest_alert['region']}")
        print(f"Message: {latest_alert['message']}")
        print("--------------------")

except requests.exceptions.RequestException as e:
    print(f"An error occurred while trying to fetch the website: {e}")



--- Latest Alert ---
Time:    24-06-2025 16:06:19
Service: Ambulance
Region:  Gooi en Vechtstreek
Message: A2 Almere 112945
--------------------
